In [1]:
import pandas as pd
import os
import git
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics
import pymc3 as pm
import theano.tensor as tt

def get_git_root(path):

        git_repo = git.Repo(path, search_parent_directories=True)
        #git_root = git_repo.git.rev_parse("--show-toplevel")
        
        return git_repo.working_dir

In [2]:
top_level_git_dir = get_git_root(os.getcwd())
raw_data_dir = os.path.join(top_level_git_dir, "data", "raw")

train_csv_path = os.path.join(raw_data_dir, "train_values.csv")
test_csv_path = os.path.join(raw_data_dir, "test_values.csv")
train_labels_csv_path = os.path.join(raw_data_dir, "train_labels.csv")
submission_format_csv_path = os.path.join(raw_data_dir, "submission_format.csv")

train_df = pd.read_csv(train_csv_path, index_col = "row_id")
train_labels_df = pd.read_csv(train_labels_csv_path, index_col = "process_id")
test_df = pd.read_csv(test_csv_path, index_col = "row_id")
submission_format_df = pd.read_csv(submission_format_csv_path, index_col = "process_id")


/Users/greg.bolla/.local/share/virtualenvs/driven-data-rinse-over-run-w5STMFPe/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
def prep_full_df(df):

    df["timestamp"] = pd.to_datetime(df["timestamp"])
    
    df = df.assign(turbidity_in_liters = \
        np.maximum(0, df.return_flow) * df.return_turbidity)

    df['process_phase'] = df.process_id.astype(str) + "_" + df.phase.astype(str)
    df = df[df.phase != "final_rinse"]
    
    return df

In [4]:
def prep_metadata(df):
    meta_df = df[["process_id", "pipeline"]].drop_duplicates().set_index("process_id")
    meta_df = pd.get_dummies(meta_df)
    
    if 'L12' not in meta_df.columns:
        meta_df['pipeline_L12'] = False
    
    for col in meta_df.columns:
        if "pipeline" in col:
            meta_df[col] = meta_df[col].astype(bool)
            
    meta_df["num_phases"] = df.groupby("process_id")["phase"].apply(lambda x: x.nunique())
    
    return meta_df

In [5]:
ts_cols = [
    'process_id',
    'timestamp',
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid',
    "turbidity_in_liters"
]

def prep_time_series_features(df, columns = None):
    
    if columns is None:
        columns = df.columns
    
    df = df.sort_values(by=["process_id", "timestamp"], ascending=True)
    process_duration_ts = df.groupby('process_id')["timestamp"].max() - df.groupby('process_id')["timestamp"].min() 
    process_duration_ts = process_duration_ts.rename('process_duration')
    process_duration = process_duration_ts.apply(lambda row: row.total_seconds())
    
    ts_df = df[ts_cols].set_index('process_id')
    
    # define fxn before calling in .agg to make col name more descriptive (in place of <lambda>)
    def last_five_mean(x):
        return x.tail(5).mean()
    
    ts_features_agg_df = ts_df.groupby('process_id').agg(['min', 'max', 'mean', 'std', last_five_mean])
    
    ts_features_df = pd.concat([process_duration, ts_features_agg_df], axis = 1)
    return ts_features_df

In [6]:
def prep_dummy_vars(df):
    
    categorical_cols = ["num_phases"]

    for cat_col in categorical_cols:
        dummy_df = pd.get_dummies(df[cat_col], prefix=cat_col, dummy_na = False)
        dummy_df = dummy_df.astype('bool')

        drop_val = df.groupby([cat_col]).size().idxmax()

        drop_col = "{}_{}".format(cat_col, drop_val)
        df = pd.concat([df, dummy_df], axis=1)
        df = df.drop(drop_col, axis=1)    
        df[cat_col] = df[cat_col].astype(object)
    
    return df

In [7]:
def clean_feature_df(df):
    
    new_col_names = []
    for col in df.columns.ravel():
        if isinstance(col, str):
            new_col_names.append(col)
        elif isinstance(col, tuple):
            col_name = "{}_{}".format(col[0], col[1])
            new_col_names.append(col_name)
    df.columns = new_col_names
    
    return df

In [8]:
def create_feature_matrix(df):
    
    prepped_df = prep_full_df(df)
    metadata_df = prep_metadata(prepped_df)
    time_series_df = prep_time_series_features(prepped_df)
    
    dfs_to_concat = [metadata_df, time_series_df]
    
    feature_df = pd.concat(dfs_to_concat, axis=1)
    feature_df = prep_dummy_vars(feature_df)
    
    df_to_return = clean_feature_df(feature_df)

    return df_to_return

In [9]:
train_features_df = create_feature_matrix(train_df)

indices_to_keep = list(set(train_features_df.index).intersection(set(train_labels_df.index)))
# figure out why 16 indices dropped out of train_features_df
train_labels_df = train_labels_df[train_labels_df.index.isin(indices_to_keep)]
train_features_w_response = train_features_df.join(train_labels_df)
train_features_w_response.head()

,pipeline_L1,pipeline_L10,pipeline_L11,pipeline_L12,pipeline_L2,pipeline_L3,pipeline_L4,pipeline_L6,pipeline_L7,pipeline_L8,...,tank_concentration_acid_last_five_mean,turbidity_in_liters_min,turbidity_in_liters_max,turbidity_in_liters_mean,turbidity_in_liters_std,turbidity_in_liters_last_five_mean,num_phases_1,num_phases_2,num_phases_3,final_rinse_total_turbidity_liter
process_id,,,,,,,,,,,,,,,,,,,,,
20001,False,False,False,False,False,False,True,False,False,False,...,44.653038,818.406942,1.579919e+06,105488.460366,174650.861160,30300.051942,False,False,False,4.318275e+06
20002,False,False,False,False,False,True,False,False,False,False,...,44.229616,499.442792,2.976941e+06,854203.372900,563689.439444,949644.159635,False,True,False,4.375286e+05
20003,False,False,False,False,False,True,False,False,False,False,...,44.716846,152.522484,1.431140e+06,44218.000816,127420.220308,5287.641592,False,False,False,4.271977e+05
20004,False,False,False,False,False,False,False,False,True,False,...,45.226021,0.000000,3.162818e+06,212923.854423,387856.686586,22306.533910,False,False,False,7.197830e+05
20005,False,False,False,False,False,False,False,False,True,False,...,43.952939,0.000000,2.066256e+05,23587.698324,26813.228206,45723.010454,True,False,False,4.133107e+05


In [10]:
response_var = ["final_rinse_total_turbidity_liter"]
pred_df = train_features_w_response.drop(response_var, axis=1)
response_df = train_features_w_response[response_var]

pred_train, pred_test, response_train, response_test = train_test_split(pred_df, response_df, test_size=0.01, random_state=223)



In [11]:
def adj_r2_score(lm, y, y_pred):
    adj_r2 = 1 - float(len(y)-1)/(len(y)-len(lm.coef_)-1)*(1 - metrics.r2_score(y,y_pred))
    return adj_r2

In [12]:
def calc_mape(pred_array, actual_array):
    
    threshold = 290000

    
    mape_array = []
    for pred, actual in zip(pred_array, actual_array.values):
        #print("{} - {}".format(pred[0], type(pred[0])))
        #print("{} - {}".format(actual[0], type(actual[0])))
        mape = (abs(pred - actual) / max(abs(actual), threshold))
        mape_array.append(mape)
        
    return mape_array


In [13]:
pred_train["turbidity_in_liters_mean"].head()

process_id
24413      3648.224135
27841     91667.214931
22114    124728.348050
22149      3966.718011
25281    191481.966164
Name: turbidity_in_liters_mean, dtype: float64

In [42]:
import matplotlib.pyplot as pyplot
#pyplot.hist(response_df, bins=5, label=response_var)
x1_var = pred_train["turbidity_in_liters_mean"]

np.std(pred_train["turbidity_in_liters_mean"])

148691.19693241388

In [15]:
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1
beta = [1, 2.5]

# Size of dataset
size = 100

# Predictor variable
X1 = np.random.randn(size)
X2 = np.random.randn(size) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(size)*sigma
Y

array([ 9.38706859e-01,  4.10296149e-01,  3.83981292e+00,  1.48115418e+00,
        4.02779506e-01,  2.46184530e+00, -1.42342679e+00, -1.27520755e+00,
        2.38380704e+00, -3.90761758e-01,  6.86815665e-01,  2.10641559e+00,
        1.84890360e+00, -8.04359754e-01,  3.93284941e-01,  2.31721220e+00,
        3.41651416e+00,  3.39016804e+00,  2.22246532e+00,  3.77308673e-01,
        3.43806883e-01,  1.66274112e+00, -1.20663529e-01,  2.18829692e+00,
        1.50706675e+00, -1.19159361e+00,  1.44784359e+00, -1.55349860e+00,
       -1.40248284e-01, -1.96609652e-02, -1.35472064e+00, -1.59474188e+00,
       -1.39656749e+00,  5.29754386e-01,  2.63051387e+00,  5.53932221e-01,
        1.76084808e+00,  2.39686504e+00,  1.47396672e+00,  9.07514885e-01,
        7.37921664e-02, -3.82899347e-01,  1.49271947e+00,  7.65880501e-01,
        2.05273917e+00,  5.63172455e-01,  4.25098874e+00,  3.26909416e-02,
        3.93785393e-01,  3.67324277e+00,  1.69575050e+00,  9.38133214e-01,
        1.35531685e+00, -

In [51]:
# find prior of response variable by simply looking at the mean/sd of response of train
response_var = "final_rinse_total_turbidity_liter"
x1_var = pred_train["turbidity_in_liters_mean"]
std = np.std(response_train[response_var])

data = dict(x=pred_train["turbidity_in_liters_mean"], y=response_train["final_rinse_total_turbidity_liter"])
# will use half normal as the distribution. pareto may also be appropriate
with pm.Model() as turbidity_model: 
    
#     x = pm.HalfNormal('x', sd=150000, observed=pred_train["turbidity_in_liters_mean"])
#     intercept = pm.HalfNormal('intercept', sd = 5000000, shape=len(response_train))
#     coef = pm.Normal('coef', mu = 0, sd=1, shape=len(response_train))
#     error = pm.Normal('error', mu=-500000, sd=7000000)
    
    #mu = intercept + (coef * x1_var)
    
    pm.glm.GLM.from_formula('y ~ x', data)
    trace = pm.sample(10)
    #y_obs = pm.HalfNormal('y_obs', sd=error, observed=response_train["final_rinse_total_turbidity_liter"].values)

Only 10 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sd, x, Intercept]
Sampling 2 chains:   0%|          | 0/1020 [00:00<?, ?draws/s]


ValueError: Not enough samples to build a trace.

In [50]:
#response_train["final_rinse_total_turbidity_liter"].values
#turbidity_model.check_test_point()
turbidity_model.observed_RVs

[x]

In [48]:
for RV in turbidity_model.basic_RVs:
    print(RV.name, RV.logp(turbidity_model.test_point))

intercept_log__ -3814.0478981586543
coef -4552.4214934961055
error -16.68035924022426
x -63721.532288925555
y_obs -inf


In [29]:
with turbidity_model:
    trace = pm.sample(10)

Only 10 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [error, coef, intercept]
Sampling 2 chains:   0%|          | 0/1020 [00:00<?, ?draws/s]/Users/greg.bolla/.local/share/virtualenvs/driven-data-rinse-over-run-w5STMFPe/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/greg.bolla/.local/share/virtualenvs/driven-data-rinse-over-run-w5STMFPe/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)

Bad initial energy, check any log probabilities that are inf or -inf, nan or very small:
y_obs   -inf


ParallelSamplingError: Bad initial energy